In [4]:
# Importing necessary libraries
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split

# Load train and test datasets
train_essays = pd.read_csv('/kaggle/input/llm-detection-dataset/train_essays.csv')
test_essays = pd.read_csv('/kaggle/input/llm-detection-dataset/test_essays.csv')

# Preprocessing the data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess_data(texts):
    return tokenizer(
        texts,
        padding='max_length',
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )

class EssaysDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        # Ensure labels are a list of integers
        self.labels = labels if isinstance(labels, list) else labels.tolist()

    def __getitem__(self, idx):
        item = {key: val[idx].detach().clone() for key, val in self.encodings.items()}
        # Convert integer label to a tensor of dtype long
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

# Preprocess and split the train dataset
X_train, X_val, y_train, y_val = train_test_split(train_essays['text'], train_essays['generated'], test_size=0.2)
train_encodings = preprocess_data(X_train.tolist())
val_encodings = preprocess_data(X_val.tolist())

train_dataset = EssaysDataset(train_encodings, y_train.tolist())
val_dataset = EssaysDataset(val_encodings, y_val.tolist())

# Load BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model
trainer.train()



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,0.511600
20,0.436600
30,0.342600
40,0.245000
50,0.169000
60,0.079500
70,0.037200
80,0.061800
90,0.010800
100,0.007200


In [5]:
# Evaluate the model
trainer.evaluate()

# Preprocess the test dataset
test_encodings = preprocess_data(test_essays['text'].tolist())
test_dataset = EssaysDataset(test_encodings, torch.zeros(len(test_essays), dtype=torch.long))
# Predictions
predictions = trainer.predict(test_dataset)

# Convert predictions to binary labels
pred_labels = predictions.predictions.argmax(axis=-1)

# Prepare the submission file
submission = test_essays[['id']].copy()
submission['generated'] = pred_labels

# Save submission file
submission.to_csv('submission.csv', index=False)